# Import các thư viện cần thiết

In [1]:
from google.cloud import bigquery
import pandas as pd
from dotenv import load_dotenv
import os

d:\Python\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


# Load môi trường (file .env bao gồm project_id và đường dẫn tới file JSON)

In [2]:
load_dotenv()
BIGQUERY_KEY_PATH = os.getenv("BIGQUERY_PATH")
PROJECT_ID = os.getenv("BIGQUERY_PROJECT_ID")

In [3]:
# Kiểm tra nhanh để đảm bảo các biến đã được tải
if not BIGQUERY_KEY_PATH or not PROJECT_ID:
    print("🚨 Lỗi: Không tìm thấy biến môi trường 'BIGQUERY_PATH' hoặc 'PROJECT_ID'.")
    print("Vui lòng kiểm tra lại file .env.")
    exit()

In [4]:
client = bigquery.Client.from_service_account_json(BIGQUERY_KEY_PATH, project = PROJECT_ID)

In [5]:
CSV_PATH = "eth_onchain_features.csv"

# Update Features

In [6]:
def update_feature(query: str):
    print("--- Bắt đầu cập nhật các feature mới ---")

    # 1. Load CSV cũ (df)
    if os.path.exists(CSV_PATH):
        df_old = pd.read_csv(CSV_PATH)
        df_old['date'] = pd.to_datetime(df_old['date']) # Đảm bảo cột date là kiểu datetime
    else:
        df_old = pd.DataFrame() # Đổi tên thành df_old

    # 2. Query BigQuery → lấy DataFrame mới (new_df)
    try:
        query_job = client.query(query)

        results_list = []

        for row in query_job.result():
            results_list.append(dict(row))

        # SỬA LỖI QUAN TRỌNG: Gán kết quả vào biến new_df
        new_df = pd.DataFrame(results_list)

        print("🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!")

    except Exception as e:
        print(f"❌ Lỗi truy vấn BigQuery: {e}")
        return

    # 3. Xử lý DataFrame mới (new_df)

    # Chuyển cột date thành kiểu datetime
    if 'date' not in new_df.columns:
        raise ValueError("Truy vấn SQL phải trả về cột 'date'!")

    new_df['date'] = pd.to_datetime(new_df['date'])

    new_feature_names = [c for c in new_df.columns if c != 'date']
    print(f"Đã lấy được các feature: {', '.join(new_feature_names)}")

    # 4. Merge vào file CSV cũ dựa trên cột date
    if df_old.empty: # Dùng df_old
        final_df = new_df
    else:
        # Merge dữ liệu cũ (df_old) và dữ liệu mới (new_df)
        final_df = pd.merge(df_old, new_df, on="date", how="outer")

    # 5. Xóa trùng và sắp xếp
    final_df = final_df.drop_duplicates(subset=["date"], keep='last').sort_values("date").reset_index(drop=True)

    # 6. Lưu lại
    final_df.to_csv(CSV_PATH, index=False)
    print(f"✅ Đã update CSV thành công! (Tổng dòng: {len(final_df)})")

# Bắt đầu thực hiện query 

# Query daily_transaction

In [14]:
query = """
    SELECT
      DATE(block_timestamp) AS date,
      COUNT(*) AS transaction_count
    FROM `bigquery-public-data.crypto_ethereum.transactions`
    WHERE block_timestamp >= TIMESTAMP('2022-01-01')
    GROUP BY date
    ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: transaction_count
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# Query active_address

In [ ]:
query = """
    SELECT
        DATE(block_timestamp) AS date,
        COUNT(DISTINCT from_address) + COUNT(DISTINCT to_address) AS active_addresses
    FROM `bigquery-public-data.crypto_ethereum.transactions`
    -- THÊM LỌC: Đảm bảo chỉ lấy dữ liệu trong 1 năm gần nhất
    WHERE block_timestamp >= TIMESTAMP('2022-01-01')
    GROUP BY date
    ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: active_addresses
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# New address count

In [7]:
query = """
SELECT
 DATE(t1.block_timestamp) AS date,
 COUNT(t1.address) AS new_addresses
FROM (
 -- Truy vấn con: Tìm thời điểm xuất hiện đầu tiên của MỌI địa chỉ
 SELECT
        from_address AS address,
        MIN(block_timestamp) AS block_timestamp
 FROM `bigquery-public-data.crypto_ethereum.transactions`
 WHERE from_address IS NOT NULL
 GROUP BY from_address
) AS t1
-- THÊM LỌC: Đảm bảo thời điểm xuất hiện lần đầu (block_timestamp)
-- chỉ nằm trong 1 năm gần nhất. Đây là bước tối ưu hóa QUAN TRỌNG.
WHERE t1.block_timestamp >= TIMESTAMP('2022-01-01')
-- 2. Dùng biểu thức hàm cho GROUP BY (Đã sửa lỗi)
GROUP BY DATE(t1.block_timestamp)
ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: new_addresses
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# average gas limit and average gas used and total gas used

In [12]:
query = """
SELECT
 DATE(timestamp) AS date,
 AVG(gas_used) AS avg_gas_used,
 AVG(gas_limit) AS avg_gas_limit,
 SUM(gas_used) AS total_gas_used
FROM `bigquery-public-data.crypto_ethereum.blocks`
-- THÊM LỌC: Đảm bảo chỉ lấy dữ liệu trong 1 năm gần nhất
WHERE timestamp >= TIMESTAMP('2022-01-01')
GROUP BY date
ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: avg_gas_used, avg_gas_limit, total_gas_used
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# Total eth transfer

In [8]:
query = """
SELECT
 DATE(block_timestamp) AS date,
 SUM(value / 1e18) AS eth_transferred
FROM `bigquery-public-data.crypto_ethereum.transactions`
-- Lọc theo điều kiện giá trị và thời gian
WHERE value > 0
  AND block_timestamp >= TIMESTAMP('2022-01-01')
GROUP BY date
ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: eth_transferred
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# Total eth burned

In [15]:
query = """
SELECT
 DATE(timestamp) AS date,
 SUM(CAST(base_fee_per_gas AS NUMERIC) * gas_used) / 1e18 AS eth_burned
FROM `bigquery-public-data.crypto_ethereum.blocks`
WHERE base_fee_per_gas IS NOT NULL
  -- THÊM LỌC: Đảm bảo chỉ lấy dữ liệu trong 1 năm gần nhất
  AND timestamp >= TIMESTAMP('2022-01-01')
GROUP BY date
ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: eth_burned
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# average_gas_price_and_total_fees_eth_pay_for

In [16]:
query = """
SELECT
 DATE(block_timestamp) AS date,
 AVG(gas_price / 1e9) AS avg_gas_price_gwei,
FROM `bigquery-public-data.crypto_ethereum.transactions`
-- THÊM LỌC: Đảm bảo chỉ lấy dữ liệu trong 1 năm gần nhất
WHERE block_timestamp >= TIMESTAMP('2022-01-01')
GROUP BY date
ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: avg_gas_price_gwei
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# average and Median gas price gwei

In [17]:
query = """
SELECT
 DATE(block_timestamp) AS date,
 APPROX_QUANTILES(gas_price / 1e9, 2)[OFFSET(1)] AS median_gas_price_gwei
FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE gas_price IS NOT NULL
  -- THÊM LỌC: Đảm bảo chỉ lấy dữ liệu trong 1 năm gần nhất
  AND block_timestamp >= TIMESTAMP('2022-01-01')
GROUP BY date
ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: median_gas_price_gwei
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# Daily Total fees transaction

In [18]:
query = """
SELECT
 DATE(block_timestamp) AS date,
  -- Tối ưu: Dùng CAST(gas_price AS NUMERIC) để tránh lỗi tràn số khi nhân
 SUM((CAST(gas_price AS NUMERIC) * receipt_gas_used) / 1e18) AS total_fees_eth
FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE receipt_gas_used IS NOT NULL
  -- THÊM LỌC: Đảm bảo chỉ lấy dữ liệu trong 1 năm gần nhất
  AND block_timestamp >= TIMESTAMP('2022-01-01')
GROUP BY date
ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: total_fees_eth
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# average fees transaction upon each block

In [19]:
query = """
SELECT
 DATE(block_timestamp) AS date,
  -- Tối ưu: Dùng CAST(gas_price AS NUMERIC) để tránh lỗi tràn số khi nhân
 AVG((CAST(gas_price AS NUMERIC) * receipt_gas_used) / 1e18) AS avg_tx_fee_eth
FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE receipt_gas_used IS NOT NULL
  -- THÊM LỌC: Đảm bảo chỉ lấy dữ liệu trong 1 năm gần nhất
  AND block_timestamp >= TIMESTAMP('2022-01-01')
GROUP BY date
ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: avg_tx_fee_eth
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# Lượng ETH phát hành mới (Block Reward)

In [9]:
query = """
SELECT
 DATE(t1.block_timestamp) AS date,
 SUM(
      -- Sử dụng receipt_gas_used, gas_price, và base_fee_per_gas từ blocks
      CAST(t1.receipt_gas_used AS NUMERIC) * (CAST(t1.gas_price AS NUMERIC) - CAST(t2.base_fee_per_gas AS NUMERIC))
  ) / 1e18 AS eth_issued_as_validator_rewards
FROM `bigquery-public-data.crypto_ethereum.transactions` AS t1
INNER JOIN `bigquery-public-data.crypto_ethereum.blocks` AS t2
    -- Ghép hai bảng bằng số khối (block_number)
    ON t1.block_number = t2.number
WHERE t2.base_fee_per_gas IS NOT NULL
  AND t1.block_timestamp >= TIMESTAMP('2022-01-01')
GROUP BY date
ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: eth_issued_as_validator_rewards
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# Net issuance ( Phát hành ròng)

In [10]:
query = """
WITH issued AS (
  -- Tính ETH Issued: Tổng Phí Ưu Tiên (Validator Rewards)
  SELECT
    DATE(t1.block_timestamp) AS date,
    SUM(
        -- Phí ưu tiên = (gas_price - base_fee_per_gas) * receipt_gas_used
        -- Chúng ta phải JOIN để lấy base_fee_per_gas từ bảng blocks (t2)
        CAST(t1.receipt_gas_used AS NUMERIC) * (CAST(t1.gas_price AS NUMERIC) - CAST(t2.base_fee_per_gas AS NUMERIC))
    ) / 1e18 AS eth_issued
  FROM `bigquery-public-data.crypto_ethereum.transactions` AS t1
  INNER JOIN `bigquery-public-data.crypto_ethereum.blocks` AS t2
    -- Ghép hai bảng bằng số khối
    ON t1.block_number = t2.number
  WHERE t2.base_fee_per_gas IS NOT NULL -- Chỉ tính từ các khối EIP-1559 trở đi
    AND t1.block_timestamp >= TIMESTAMP('2022-01-01')
  GROUP BY date
),
burned AS (
  -- Tính ETH Burned: Tổng Phí Cơ Sở (Base Fees)
  SELECT
    DATE(timestamp) AS date,
    -- ETH bị đốt = base_fee_per_gas * gas_used (từ bảng blocks)
    SUM(CAST(base_fee_per_gas AS NUMERIC) * gas_used) / 1e18 AS eth_burned
  FROM `bigquery-public-data.crypto_ethereum.blocks`
  WHERE base_fee_per_gas IS NOT NULL
    AND timestamp >= TIMESTAMP('2022-01-01')
  GROUP BY date
)
SELECT
 i.date,
 i.eth_issued,
 (i.eth_issued - b.eth_burned) AS net_issuance
FROM issued i
-- Dùng INNER JOIN để chỉ so sánh các ngày có dữ liệu đầy đủ
INNER JOIN burned b ON i.date = b.date
ORDER BY i.date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: eth_issued, net_issuance
✅ Đã update CSV thành công! (Tổng dòng: 1438)


# Phần thưởng validator mỗi ngày (giống Lượng ETH phát hành trong kỷ nguyên PoS)

In [11]:
query = """
WITH issued AS (
  -- Tính ETH Issued: Tổng Phí Ưu Tiên (Validator Rewards)
  SELECT
    DATE(t1.block_timestamp) AS date,
    SUM(
        -- Phí ưu tiên = (gas_price - base_fee_per_gas) * receipt_gas_used
        CAST(t1.receipt_gas_used AS NUMERIC) * (CAST(t1.gas_price AS NUMERIC) - CAST(t2.base_fee_per_gas AS NUMERIC))
    ) / 1e18 AS validator_rewards_eth
  FROM `bigquery-public-data.crypto_ethereum.transactions` AS t1
  INNER JOIN `bigquery-public-data.crypto_ethereum.blocks` AS t2
    ON t1.block_number = t2.number
  WHERE t2.base_fee_per_gas IS NOT NULL
    AND t1.block_timestamp >= TIMESTAMP('2022-01-01')
  GROUP BY date
)
SELECT
    date,
    validator_rewards_eth
FROM issued
ORDER BY date;
"""
update_feature(query)

--- Bắt đầu cập nhật các feature mới ---
🎉 Truy vấn và tạo DataFrame mới (new_df) thành công!
Đã lấy được các feature: validator_rewards_eth
✅ Đã update CSV thành công! (Tổng dòng: 1438)
